In [ ]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [41]:
conversation = [
    {"role": "user", "parts": ["Hi my name is Akshay, here after everytime you talk to me, please talk from First person POV and address me only by my name"]},  
    {"role": "model", "parts": ["Ok got it Akshay!"]},  
    {"role": "user", "parts": ["How are you today"]},
    {"role": "user", "parts": [f"Here is a JSON table extracted from the PDF:\n{table_json}\n\nYou can use this data to answer my questions."]},
    {"role": "user", "parts": ["What is this your inference about the blind people?"]}
]

In [38]:
conversation

[{'role': 'user',
  'parts': ['Hi my name is Akshay, here after everytime you talk to me, please talk from First person POV and address me only by my name']},
 {'role': 'model', 'parts': ['Ok got it Akshay!']},
 {'role': 'user', 'parts': ['How are you today']},
 {'role': 'user',
  'parts': ['Here is a JSON table extracted from the PDF:\n[\n  {\n    "0": "Disability \\nCategory",\n    "1": "Participants",\n    "2": "Ballots \\nCompleted",\n    "3": "Ballots \\nIncomplete/ \\nTerminated",\n    "4": "Results",\n    "5": ""\n  },\n  {\n    "0": "",\n    "1": "",\n    "2": "",\n    "3": "",\n    "4": "Accuracy",\n    "5": "Time to \\ncomplete"\n  },\n  {\n    "0": "Blind",\n    "1": "5",\n    "2": "1",\n    "3": "4",\n    "4": "34.5%, n=1",\n    "5": "1199 sec, n=1"\n  },\n  {\n    "0": "Low Vision",\n    "1": "5",\n    "2": "2",\n    "3": "3",\n    "4": "98.3% n=2 \\n(97.7%, n=3)",\n    "5": "1716 sec, n=3 \\n(1934 sec, n=2)"\n  },\n  {\n    "0": "Dexterity",\n    "1": "5",\n    "2": "4",\

In [ ]:
import google.generativeai as genai
import pandas as pd
import pdfplumber
import camelot
import json
genai.configure(api_key=GOOGLE_API_KEY)  


model = genai.GenerativeModel("gemini-2.0-flash")


In [42]:
response = model.generate_content(conversation)

print(response.text)

Akshay, based on the data I have, I see that there were 5 participants in the Blind category, but only 1 ballot was completed. The accuracy for that single ballot was 34.5%, and it took 1199 seconds to complete.



In [22]:
#Excel
df = pd.read_excel("Financial Sample.xlsx")
data = df.values.tolist()

In [24]:
#pdf
with pdfplumber.open("table.pdf") as f:
    for i in f.pages:
        print(i.extract_tables())

[[['', None, None, '', None, None, '', None, None, '', None, None, '', 'Results', None, None, None, ''], [None, None, None, None, None, None, None, None, None, '', 'Ballots', '', None, None, None, None, None, None], ['', 'Disability', '', None, None, None, '', 'Ballots', '', None, None, None, None, None, None, None, None, None], [None, None, None, '', 'Participants', '', None, None, None, None, 'Incomplete/', None, None, None, None, None, None, None], [None, 'Category', None, None, None, None, None, 'Completed', None, None, None, None, '', 'Accuracy', '', '', 'Time to', ''], [None, None, None, None, None, None, None, None, None, None, 'Terminated', None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'complete', None], [None, None, None, '', None, None, None, None, None, None, None, None, None, None, None, None, None, None], ['', None, None, None, None, None, '', None, None, None, None, None, '', Non

In [36]:
#PDF
tables = camelot.read_pdf("table.pdf", pages="1")
table_dict_list = tables[0].df.to_dict(orient="records")  # Convert to list of dictionaries

# Convert to JSON
table_json = json.dumps(table_dict_list, indent=2)

print(table_json)

[
  {
    "0": "Disability \nCategory",
    "1": "Participants",
    "2": "Ballots \nCompleted",
    "3": "Ballots \nIncomplete/ \nTerminated",
    "4": "Results",
    "5": ""
  },
  {
    "0": "",
    "1": "",
    "2": "",
    "3": "",
    "4": "Accuracy",
    "5": "Time to \ncomplete"
  },
  {
    "0": "Blind",
    "1": "5",
    "2": "1",
    "3": "4",
    "4": "34.5%, n=1",
    "5": "1199 sec, n=1"
  },
  {
    "0": "Low Vision",
    "1": "5",
    "2": "2",
    "3": "3",
    "4": "98.3% n=2 \n(97.7%, n=3)",
    "5": "1716 sec, n=3 \n(1934 sec, n=2)"
  },
  {
    "0": "Dexterity",
    "1": "5",
    "2": "4",
    "3": "1",
    "4": "98.3%, n=4",
    "5": "1672.1 sec, n=4"
  },
  {
    "0": "Mobility",
    "1": "3",
    "2": "3",
    "3": "0",
    "4": "95.4%, n=3",
    "5": "1416 sec, n=3"
  }
]


From here wrinting the code for complete program

In [ ]:
def api_call(prompt):
    conversation.append({"role": "user", "parts": [prompt] })
    response = model.generate_content(conversation)
    final_output = response.text
    conversation.append({"role": "assistant", "parts": [final_output]})
    print(final_output)

In [46]:
def init_conversation():
    global conversation
    conversation=[{"role": "user", "parts": ['''Your name is StrataBot. You are an intelligent chatbot designed to summarize structured data and provide instant query-based insights. 
            Whether you're dealing with databases, spreadsheets, or APIs, you help users extract meaningful information effortlessly. 
            Stay in character and always introduce yourself as StrataBot when responding.''']},  
    ]

In [47]:
def new_file(filename):
    if '.pdf' in filename:
        #PDF
        tables = camelot.read_pdf("table.pdf", pages="1")
        table_dict_list = tables[0].df.to_dict(orient="records")  # Convert to list of dictionaries

        # Convert to JSON
        table_json = json.dumps(table_dict_list, indent=2)
        conversation.append({"role": "user", "parts": [f"Here is a JSON table extracted from the PDF named '{filename}':\n{table_json}\n\nYou can use this data to answer my questions. Also remember the filename as i might query with reference to the filename aswell"]})
    if ('.xlsx' in filename) or ('.xls' in filename):
        df = pd.read_excel("Financial Sample.xlsx")
        data_json = json.dumps(df.to_dict(orient="records"), indent=2)
        conversation.append({"role": "user", "parts": [f"Here is a JSON table extracted from the Excel named '{filename}':\n{table_json}\n\nYou can use this data to answer my questions. Also remember the filename as i might query with reference to the filename aswell"]})


        
